<a href="https://colab.research.google.com/github/C23-PS435-bangkit/MachineLearning/blob/haah/intial_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libs

In [ ]:
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import Split Datasets

check the datasets

In [ ]:
import os

base_dir = '/content/drive/MyDrive/Datasets/Bangkit - Capstone/dataset'

bd_dir = os.path.join(base_dir, 'bacterial_dermatosis')
fi_dir = os.path.join(base_dir, 'fungal_infection')
ha_dir = os.path.join(base_dir, 'hypersensitivity_allergic_dermatosis')
he_dir = os.path.join(base_dir, 'healthy')

print('total training bacterial_dermatosis images:', len(os.listdir(bd_dir)))
print('total training fungal_infection images:', len(os.listdir(fi_dir)))
print('total training hypersensitivity_allergic_dermatosis images:', len(os.listdir(ha_dir)))
print('total training healthy images:', len(os.listdir(he_dir)))

bd_files = os.listdir(bd_dir)
print(bd_files[:10])

fi_files = os.listdir(fi_dir)
print(fi_files[:10])

ha_files = os.listdir(ha_dir)
print(ha_files[:10])

he_files = os.listdir(he_dir)
print(he_files[:10])

total training bacterial_dermatosis images: 121
total training fungal_infection images: 110
total training hypersensitivity_allergic_dermatosis images: 130
total training healthy images: 260
['dog210612_03_01_13_pic1.jpg', 'dog210612_03_01_13_pic6.jpg', 'dog210612_03_01_13_pic0.jpg', 'dog210612_03_01_13_pic5.jpg', 'dog210612_03_01_13_pic3.jpg', 'dog210612_03_01_13_pic7.jpg', 'dog210612_03_01_13_pic9.jpg', 'dog210612_03_01_13_pic8.jpg', 'dog210612_03_01_13_pic2.jpg', 'dog210612_03_01_13_pic4.jpg']
['dog210422_04_02_33_pic7.jpg', 'dog210422_04_02_33_pic0.jpg', 'dog210422_04_02_33_pic6.jpg', 'dog210422_04_02_33_pic8.jpg', 'dog210422_04_02_33_pic5.jpg', 'dog210422_04_02_33_pic9.jpg', 'dog210422_04_02_33_pic4.jpg', 'dog210422_04_02_33_pic2.jpg', 'dog210422_04_02_33_pic3.jpg', 'dog210422_04_02_33_pic1.jpg']
['dog210424_05_01_31_pic0.jpg', 'dog210424_05_01_31_pic6.jpg', 'dog210424_05_01_31_pic1.jpg', 'dog210424_05_01_31_pic8.jpg', 'dog210424_05_01_31_pic4.jpg', 'dog210424_05_01_31_pic2.jpg', 

In [ ]:
root_dir = '/content/drive/MyDrive/Datasets/Bangkit - Capstone/dataset/arranged'

if os.path.exists(root_dir):
  shutil.rmtree(root_dir)

def create_train_val_dirs(root_path):

  # List of labels
  labels = ['bd', 'fi', 'ha', 'he']

  # Create train directory
  train_dir = os.path.join(root_path, 'training')
  os.makedirs(train_dir, exist_ok=True)

  for label in labels:
      label_dir = os.path.join(train_dir, label)
      os.makedirs(label_dir, exist_ok=True)

  # Create validation directory
  val_dir = os.path.join(root_path, 'validation')
  os.makedirs(val_dir, exist_ok=True)

  for label in labels:
      label_dir = os.path.join(val_dir, label)
      os.makedirs(label_dir, exist_ok=True)

  
try:
  create_train_val_dirs(root_path=root_dir)
except FileExistsError:
  print("Error")

In [ ]:
for rootdir, dirs, files in os.walk(root_dir):
    for subdir in dirs:
        print(os.path.join(rootdir, subdir))

/content/drive/MyDrive/Datasets/Bangkit - Capstone/dataset/arranged/training
/content/drive/MyDrive/Datasets/Bangkit - Capstone/dataset/arranged/validation
/content/drive/MyDrive/Datasets/Bangkit - Capstone/dataset/arranged/training/bd
/content/drive/MyDrive/Datasets/Bangkit - Capstone/dataset/arranged/training/fi
/content/drive/MyDrive/Datasets/Bangkit - Capstone/dataset/arranged/training/ha
/content/drive/MyDrive/Datasets/Bangkit - Capstone/dataset/arranged/training/he
/content/drive/MyDrive/Datasets/Bangkit - Capstone/dataset/arranged/validation/bd
/content/drive/MyDrive/Datasets/Bangkit - Capstone/dataset/arranged/validation/fi
/content/drive/MyDrive/Datasets/Bangkit - Capstone/dataset/arranged/validation/ha
/content/drive/MyDrive/Datasets/Bangkit - Capstone/dataset/arranged/validation/he


## Old Function

In [ ]:
def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):
  files = os.listdir(SOURCE_DIR)

  verified_files = []
  for file in files :
    if os.path.getsize(os.path.join(SOURCE_DIR, file)) == 0 :
      print(f"{file} is zero length, so ignoring.")
    else :
      verified_files.append(file)
      

  # shuffle the files
  random.shuffle(verified_files)

  # calculate the split index
  split_idx = int(SPLIT_SIZE * len(verified_files))

  # split the files
  train_files = verified_files[:split_idx]
  val_files = verified_files[split_idx:]

  # copy train files
  for file in train_files:
      src_path = os.path.join(SOURCE_DIR, file)
      dst_path = os.path.join(TRAINING_DIR, file)
      shutil.copy(src_path, dst_path)

  # copy validation files
  for file in val_files:
      src_path = os.path.join(SOURCE_DIR, file)
      dst_path = os.path.join(VALIDATION_DIR, file)
      shutil.copy(src_path, dst_path)

## Try to Crop

In [ ]:
# function to crop image
# note: the amount of croped pixels were approxed by me (nopal) 
#       by measure the excess pixels
def crop_image(image):
    crop_width = 1920 - 170 - 160  # Calculate the resulting width after trimming
    crop_height = 1080  # Height remains the same
    crop_location = (170, 0)  # Starting position of the crop
    cropped_image = tf.image.crop(image, crop_location, [crop_height, crop_width])
    return cropped_image

In [ ]:
def crop_image(image):
    crop_width = 1920 - 170 - 150  # Calculate the resulting width after trimming
    crop_height = 1080  # Height remains the same
    crop_location = (170, 0)  # Starting position of the crop

    # Crop the image using array indexing
    cropped_image = image[crop_location[1]:crop_location[1] + crop_height,
                          crop_location[0]:crop_location[0] + crop_width, :]

    return cropped_image

In [ ]:
# function to split dataset into training and validation
def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):
    files = os.listdir(SOURCE_DIR)

    verified_files = []
    for file in files:
        if os.path.getsize(os.path.join(SOURCE_DIR, file)) == 0:
            print(f"{file} is zero length, so ignoring.")
        else:
            verified_files.append(file)

    # Shuffle the files
    random.shuffle(verified_files)

    # Calculate the split index
    split_idx = int(SPLIT_SIZE * len(verified_files))

    # Split the files
    train_files = verified_files[:split_idx]
    val_files = verified_files[split_idx:]

    # Copy train files
    for file in train_files:
        src_path = os.path.join(SOURCE_DIR, file)
        dst_path = os.path.join(TRAINING_DIR, file)
        # Load the image using TensorFlow
        image = tf.io.read_file(src_path)
        image = tf.image.decode_image(image)
        # Crop the image
        cropped_image = crop_image(image)
        # Save the cropped image to the destination directory
        tf.io.write_file(dst_path, tf.image.encode_png(cropped_image))

    # Copy validation files
    for file in val_files:
        src_path = os.path.join(SOURCE_DIR, file)
        dst_path = os.path.join(VALIDATION_DIR, file)
        # Load the image using TensorFlow
        image = tf.io.read_file(src_path)
        image = tf.image.decode_image(image)
        # Crop the image
        cropped_image = crop_image(image)
        # Save the cropped image to the destination directory
        tf.io.write_file(dst_path, tf.image.encode_png(cropped_image))

In [ ]:
TRAINING_DIR = "/content/drive/MyDrive/Datasets/Bangkit - Capstone/dataset/arranged/training/"
VALIDATION_DIR = "/content/drive/MyDrive/Datasets/Bangkit - Capstone/dataset/arranged/validation/"

LABELS = ['bd', 'fi', 'ha', 'he']

TRAINING_LABEL_DIRS = [os.path.join(TRAINING_DIR, label) for label in LABELS]
VALIDATION_LABEL_DIRS = [os.path.join(VALIDATION_DIR, label) for label in LABELS]

# Empty directories in case this cell runs multiple times
for label_dir in TRAINING_LABEL_DIRS:
    if len(os.listdir(label_dir)) > 0:
        for file in os.scandir(label_dir):
            os.remove(file.path)

for label_dir in VALIDATION_LABEL_DIRS:
    if len(os.listdir(label_dir)) > 0:
        for file in os.scandir(label_dir):
            os.remove(file.path)

# Define proportion of images used for training
split_size = 0.8

# Run the function
for label in LABELS:
    source_dir = eval(f'{label}_dir')
    train_dir = os.path.join(TRAINING_DIR, label)
    val_dir = os.path.join(VALIDATION_DIR, label)
    split_data(source_dir, train_dir, val_dir, split_size)

# Check the number of images in each directory

# Original source directories should contain unchanged images
for label in LABELS:
    source_dir = eval(f'{label}_dir')
    print(f"Original {label}'s directory has {len(os.listdir(source_dir))} images")

# Training and validation splits
for i, label in enumerate(LABELS):
    print(f"There are {len(os.listdir(TRAINING_LABEL_DIRS[i]))} images of {label} for training")
    print(f"There are {len(os.listdir(VALIDATION_LABEL_DIRS[i]))} images of {label} for validation")

Original bd's directory has 121 images
Original fi's directory has 110 images
Original ha's directory has 130 images
Original he's directory has 260 images
There are 96 images of bd for training
There are 25 images of bd for validation
There are 88 images of fi for training
There are 22 images of fi for validation
There are 104 images of ha for training
There are 26 images of ha for validation
There are 208 images of he for training
There are 52 images of he for validation


## Image Data Generator

In [ ]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):

  # Instantiate the ImageDataGenerator class and set the desired augmentation parameters
  train_datagen = ImageDataGenerator(
      rescale=1.0/255.,
      shear_range=0.2,
      rotation_range=360,
      horizontal_flip=True,
      vertical_flip=True
  )

  # Pass in the appropriate arguments to the flow_from_directory method
  train_generator = train_datagen.flow_from_directory(
      directory=TRAINING_DIR,
      batch_size=10,
      class_mode='categorical',
      target_size=(128, 128)
  )

  # Instantiate the ImageDataGenerator class and set the rescale parameter
  validation_datagen = ImageDataGenerator(rescale=1.0/255.)

  # Pass in the appropriate arguments to the flow_from_directory method
  validation_generator = validation_datagen.flow_from_directory(
      directory=VALIDATION_DIR,
      batch_size=10,
      class_mode='categorical',
      target_size=(128, 128)
  )

  return train_generator, validation_generator

In [ ]:
# Test the generators
train_generator, validation_generator = train_val_generators(TRAINING_DIR, VALIDATION_DIR)

Found 496 images belonging to 4 classes.
Found 125 images belonging to 4 classes.


## Try Using Restnet

In [ ]:
from tensorflow.keras.applications import ResNet50

# Load the pre-trained ResNet50 model without the top layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

94765736/94765736 [==============================] - 0s 0us/step


In [ ]:
from tensorflow.keras import models, layers

# Create a new model by adding your own classifier on top of the pre-trained ResNet model
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=10, validation_data=validation_generator)

Epoch 1/10
50/50 [==============================] - 199s 4s/step - loss: 4.5865 - accuracy: 0.2964 - val_loss: 1180.5657 - val_accuracy: 0.4160
Epoch 2/10
50/50 [==============================] - 177s 4s/step - loss: 1.4706 - accuracy: 0.3770 - val_loss: 1.3741 - val_accuracy: 0.3840
Epoch 3/10
50/50 [==============================] - 178s 4s/step - loss: 1.4034 - accuracy: 0.4052 - val_loss: 1.3587 - val_accuracy: 0.4160
Epoch 4/10
50/50 [==============================] - 178s 4s/step - loss: 1.3504 - accuracy: 0.4153 - val_loss: 1.3530 - val_accuracy: 0.4160
Epoch 5/10
50/50 [==============================] - 170s 3s/step - loss: 1.3773 - accuracy: 0.4012 - val_loss: 1.3261 - val_accuracy: 0.4160
Epoch 6/10
50/50 [==============================] - 176s 4s/step - loss: 1.3900 - accuracy: 0.4153 - val_loss: 1.3177 - val_accuracy: 0.4160
Epoch 7/10
50/50 [==============================] - 170s 3s/step - loss: 1.4502 - accuracy: 0.4052 - val_loss: 1.3271 - val_accuracy: 0.4160
Epoch 8/10

## Modelling

In [ ]:
def create_model():
  model = tf.keras.models.Sequential([ 
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(128, 128, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

  
  model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
                loss=tf.keras.losses.binary_crossentropy,
                metrics=['accuracy']) 

  return model

In [ ]:
model = create_model()

history = model.fit(train_generator,
                    epochs=100,
                    verbose=1,
                    validation_data=validation_generator)

Epoch 1/100
50/50 [==============================] - 83s 2s/step - loss: 0.5686 - accuracy: 0.3972 - val_loss: 0.5436 - val_accuracy: 0.4160
Epoch 2/100
50/50 [==============================] - 93s 2s/step - loss: 0.5454 - accuracy: 0.4194 - val_loss: 0.5413 - val_accuracy: 0.4160
Epoch 3/100
50/50 [==============================] - 74s 1s/step - loss: 0.5435 - accuracy: 0.4194 - val_loss: 0.5408 - val_accuracy: 0.4160
Epoch 4/100
50/50 [==============================] - 73s 1s/step - loss: 0.5421 - accuracy: 0.4194 - val_loss: 0.5398 - val_accuracy: 0.4160
Epoch 5/100
50/50 [==============================] - 75s 1s/step - loss: 0.5420 - accuracy: 0.4194 - val_loss: 0.5401 - val_accuracy: 0.4160
Epoch 6/100
50/50 [==============================] - 83s 2s/step - loss: 0.5408 - accuracy: 0.4194 - val_loss: 0.5407 - val_accuracy: 0.4160
Epoch 7/100
50/50 [==============================] - 83s 2s/step - loss: 0.5420 - accuracy: 0.4194 - val_loss: 0.5392 - val_accuracy: 0.4160
Epoch 8/100
5

KeyboardInterrupt: ignored

## Try to crop